In [46]:
import math

from typing import Tuple, List

import numpy as np
import pyproj
import vtk

In [47]:
def proche(a, b):
    if(abs(a-b) < 0.01):
        return True
    return False


def proche2(a, b, c):
    return proche(a, b) or proche(a, c)


In [48]:
class Converter:
    """Classe pour convertir des coordonnées depuis un espace quadrilatère de départ vers un espace carré dont les coordonnées sont comprises entre 0 et 1"""

    def __init__(self, tr: Tuple[float, float], tl: Tuple[float, float], br: Tuple[float, float], bl: Tuple[float, float]) -> None:
        """Crée un convertisseur avec les cordonnée de coin l'espace de départ pour les mapper sur des coordonnées dont les valeurs sont comprises entre 0 et 1

        Args:
            tr (Tuple[float,float]): _description_
            tl (Tuple[float,float]): _description_
            br (Tuple[float,float]): _description_
            bl (Tuple[float,float]): _description_
        """
        # mapping
        px = [bl[0], br[0], tr[0], tl[0]]
        py = [bl[1], br[1], tr[1], tl[1]]

        # compute coefficients
        A = [[1, 0, 0, 0], [1, 1, 0, 0], [1, 1, 1, 1], [1, 0, 1, 0]]
        AI = np.linalg.inv(A)
        self.a = np.matmul(AI, py)
        self.b = np.matmul(AI, px)

    def l_to_x(self, y: float, x: float, ) -> Tuple[float, float]:
        """Convertit valeurs de l'espace de départ en valeurs entre 0 and 1

        Args:
            x (float): latitude
            y (float): longitude

        Returns:
            Tuple[float,float]: coordonnées dans l'espace de départ
        """
        # quadratic equation coeffs, aa*mm^2+bb*m+cc=0
        aa = self.a[3] * self.b[2] - self.a[2] * self.b[3]
        bb = self.a[3] * self.b[0] - self.a[0] * self.b[3] + self.a[1] * self.b[2] - self.a[2] * self.b[1] + x * self.b[
            3] - y * self.a[3]
        cc = self.a[1] * self.b[0] - self.a[0] * \
            self.b[1] + x * self.b[1] - y * self.a[1]

        # compute m = (-b+sqrt(b^2-4ac))/(2a)
        det = math.sqrt(bb * bb - 4 * aa * cc)
        m = (-bb + det) / (2 * aa)

        # compute l
        l = (x - self.a[0] - self.a[2] * m) / (self.a[1] + self.a[3] * m)
        return l, m

    def x_to_l(self, l: float, m: float) -> Tuple[float, float]:
        """Convertit des coordonnées entre 0 et 1 vers l'espace de départ

        Args:
            l (float): valeur entre 0 et 1
            m (float): valeur entre 0 et 1
        Returns:
             Tuple[float, float] (latitude, longitude)
        """
        assert l >= 0
        assert l <= 1
        assert m >= 0
        assert m <= 1
        newx = self.a[0] + self.a[1] * l + self.a[2] * m + self.a[3] * l * m
        newy = self.b[0] + self.b[1] * l + self.b[2] * m + self.b[3] * l * m
        return newy, newx


def get_index(lat: float, lon: float) -> Tuple[float, float]:
    assert lat >= 60
    assert lat <= 65
    assert lon >= 10
    assert lon <= 15
    return (65-lat)*6000/(65-60), (lon-10) * 6000/(15-10)


def index_to_coord(i: int, j: int) -> Tuple[float, float]:
    assert i >= 0
    assert i <= 6000
    assert j >= 0
    assert j <= 6000
    return 60+(6000-i)/6000*5, 10+j/6000*5


In [49]:
rt90 = 'epsg:3021'  # RT90
wgs84 = 'epsg:4326'  # Global lat-lon coordinate system

rt90_to_latlon = pyproj.Transformer.from_crs(rt90, wgs84)
tl = rt90_to_latlon.transform(7022573, 1349340)
tr = rt90_to_latlon.transform(7022967, 1371573)
br = rt90_to_latlon.transform(7006362, 1371835)
bl = rt90_to_latlon.transform(7005969, 1349602)

conv = Converter(tr, tl, br, bl)

In [50]:
def validate(coord):
    a, b = conv.x_to_l(coord[0], coord[1])
    c, d = conv.l_to_x(a, b)
    print(proche(coord[0], c), proche(coord[1], d))


validate((0.0, 0.0))
validate((1.0, 0.0))
validate((0.0, 1.0))
validate((1.0, 1.0))


True True
True True
True True
True True


In [51]:
def validate(coord, min, max):
    a, b = conv.l_to_x(coord[0], coord[1])
    c, d = conv.x_to_l(a, b)
    print(proche(coord[0], c), proche(coord[1], d),
          proche2(a, min, max), proche2(b, min, max))


validate(tr, 0, 1)
validate(tl, 0, 1)
validate(br, 0, 1)
validate(bl, 0, 1)


True True True True
True True True True
True True True True
True True True True


In [52]:
def validate(coord, minX, maxX, minY, maxY):
    a, b = index_to_coord(coord[0], coord[1])
    c, d = get_index(a, b)
    print(proche(coord[0], c), proche(coord[1], d),
          proche2(a, minX, maxX), proche2(b, minY, maxY))


validate((0.0, 0.0), 60, 65, 10, 15)
validate((6000.0, 0.0), 60, 65, 10, 15)
validate((0.0, 6000.0), 60, 65, 10, 15)
validate((6000.0, 6000.0), 60, 65, 10, 15)


True True True True
True True True True
True True True True
True True True True


In [53]:
def validate(coord, min, max):
    a, b = get_index(coord[0], coord[1])
    c, d = index_to_coord(a, b)
    print(proche(coord[0], c), proche(coord[1], d),
          proche2(a, min, max), proche2(b, min, max))


validate((65.0, 10.0), 0, 6000)
validate((60.0, 10.0), 0, 6000)
validate((65.0, 15.0), 0, 6000)
validate((60.0, 15.0), 0, 6000)


True True True True
True True True True
True True True True
True True True True
